# Instructions
This notebook is designed to run scoring on a saved model in Snowflake.
Follow the steps in each cell to perform the inference.

In [ ]:
# Import necessary libraries
from snowflake.snowpark import Session
import pandas as pd


In [ ]:
-- Create Stream for Real-Time Inference
CREATE OR REPLACE STREAM ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2 ON TABLE ANALYTICS_TOOL_KIT.PUBLIC.SCORING_TABLE SHOW_INITIAL_ROWS = TRUE APPEND_ONLY = TRUE;


In [ ]:
-- Select data from the Stream
SELECT * FROM ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2 WHERE METADATA$ACTION = 'INSERT';


In [ ]:
-- Create `inference_table` and insert record when stream is triggered

        CREATE OR REPLACE TABLE ANALYTICS_TOOL_KIT.PUBLIC.inference_table (
            ID INT,
            VALUE INT
        );
        

        INSERT INTO ANALYTICS_TOOL_KIT.PUBLIC.inference_table (ID, Value)
        SELECT 1,5;
        


In [ ]:
-- Create Task for Real-Time Inference Trigger

        CREATE OR REPLACE TASK ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2_task
        WAREHOUSE = COMPUTE_WH
        SCHEDULE = '1 MINUTE'
        WHEN SYSTEM$STREAM_HAS_DATA('ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2')
        AS
        BEGIN
            -- Execute stream logic to insert a record when data changes
            INSERT INTO ANALYTICS_TOOL_KIT.PUBLIC.inference_table (ID, VALUE)
            SELECT ID, 5
            FROM ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2
            WHERE METADATA$ACTION = 'INSERT';
        END;
        


# Reminder
After creating the task, go to Snowflake, find your task under the 'Tasks' section in the UI, and click the three dots in the top right corner, and click 'Resume' button in the top right corner to start the task.